# Printing Data

In [21]:
%maven org.jfree:jfreechart:1.5.3
%maven com.opencsv:opencsv:5.5
import org.jfree.chart.ChartFactory;
import org.jfree.chart.ChartUtils;
import org.jfree.chart.JFreeChart;
import org.jfree.data.category.CategoryDataset;
import org.jfree.data.category.DefaultCategoryDataset;
import com.opencsv.CSVReader;
import java.io.File;
import java.io.IOException;
import java.io.FileReader;

public class JFreeChartExample {

    public static CategoryDataset createDataset() {
        DefaultCategoryDataset dataset = new DefaultCategoryDataset();
        

        dataset.addValue(1.0, "Series1", "Category1");
        dataset.addValue(4.0, "Series1", "Category2");
        dataset.addValue(3.0, "Series1", "Category3");
        dataset.addValue(5.0, "Series1", "Category4");

        return dataset;
    }

    public static JFreeChart createChart(CategoryDataset dataset) {
        return ChartFactory.createLineChart(
                "JFreeChart Example",
                "Category",
                "Value",
                dataset
        );
    }

    public static void saveChartAsImage(JFreeChart chart, String filePath, int width, int height) throws IOException {
        ChartUtils.saveChartAsPNG(new File(filePath), chart, width, height);
    }

    public static void main(String[] args) {
        try {
            String[] tickers = ["AAPL", "AMZN", "COST", "GOOGL", "LMT", "META", "MSFT", "NOC", "TSLA", "UNH", "WMT"]
            for (String ticker : tickers) {
                CategoryDataset dataset = createDataset();
                JFreeChart chart = createChart(dataset);
                String chartname = "./stock_charts/" + tickers;

                // Save chart as image
                saveChartAsImage(chart, "./jfreechart_example.png", 800, 600);

                System.out.println("Chart saved as image: jfreechart_example.png");
            }
            
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

JFreeChartExample.main(null);


Chart saved as image: jfreechart_example.png


# Testing code for LSTM

In [4]:
%maven org.deeplearning4j:deeplearning4j-core:1.0.0-M2.1
%maven org.nd4j:nd4j-native:1.0.0-M2.1

import org.deeplearning4j.datasets.iterator.impl.*;
import org.deeplearning4j.nn.api.OptimizationAlgorithm;
import org.deeplearning4j.nn.conf.GradientNormalization;
import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
import org.deeplearning4j.nn.conf.layers.LSTM;
import org.deeplearning4j.nn.conf.layers.RnnOutputLayer;
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
import org.deeplearning4j.nn.weights.WeightInit;
import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
import org.nd4j.linalg.activations.Activation;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
import org.nd4j.linalg.learning.config.Adam;
import org.nd4j.linalg.lossfunctions.LossFunctions;
import java.util.ArrayList;
import java.util.List;

In [6]:
public class LstmExample {

    public static void main(String[] args) {
        // Define your dataset and load it
        // Example: List<double[]> features = loadFeatures();
        // List<double[]> labels = loadLabels();

        // Create DataSetIterator
        DataSetIterator iterator = new ListDataSetIterator<>(createData(features, labels));

        // Neural network configuration
        MultiLayerNetwork model = new MultiLayerNetwork(new NeuralNetConfiguration.Builder()
                .seed(123)
                .optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
                .updater(new Adam(0.01))
                .l2(1e-4)
                .weightInit(WeightInit.XAVIER)
                .gradientNormalization(GradientNormalization.ClipElementWiseAbsoluteValue)
                .gradientNormalizationThreshold(0.5)
                .list()
                .layer(new LSTM.Builder()
                        .nIn(numInputs)
                        .nOut(numHiddenUnits)
                        .activation(Activation.TANH)
                        .build())
                .layer(new RnnOutputLayer.Builder(LossFunctions.LossFunction.MSE)
                        .activation(Activation.IDENTITY)
                        .nIn(numHiddenUnits)
                        .nOut(numOutputs)
                        .build())
                .pretrain(false)
                .backprop(true)
                .build()
        );

        model.init();
        model.setListeners(new ScoreIterationListener(20));  // Print the score with every 20 iterations

        // Train the model
        int numEpochs = 50;
        for (int i = 0; i < numEpochs; i++) {
            model.fit(iterator);
        }

        // Your model is now trained and ready for prediction
    }

    private static List<DataSet> createData(List<double[]> features, List<double[]> labels) {
        List<DataSet> dataSets = new ArrayList<>();
        for (int i = 0; i < features.size(); i++) {
            dataSets.add(new DataSet(Nd4j.create(features.get(i)), Nd4j.create(labels.get(i))));
        }
        return dataSets;
    }

    // Implement methods to load your features and labels from your dataset
    // private static List<double[]> loadFeatures() { ... }
    // private static List<double[]> loadLabels() { ... }
}

CompilationException: 